# Importing libraries

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import pyarrow.parquet as pq
import pyarrow as pa
import time
import os

from sklearn.model_selection import train_test_split
import polars as pl
import datetime as dt
import gc

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

# import dash
# from dash import dcc, html
import streamlit as st
import plotly.express as px


# Import original dataset

In [4]:
original_data_1x = pd.read_csv('./all_stocks_5yr.csv')

In [5]:
original_data_1x.head()

,date,open,high,low,close,volume,name
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


# Expand data into 10X

In [6]:
# Duplicate the dataset 10 times
data_expanded_10x = pd.concat([original_data_1x] * 10, ignore_index=True)

In [7]:
# Save to a new CSV file
data_expanded_10x.to_csv("all_stocks_5yr_10x.csv", index=False)

# Expand data into 100X

In [8]:
data_expanded_100x = pd.concat([original_data_1x] * 100, ignore_index=True)
data_expanded_100x.to_csv("all_stocks_5yr_100x.csv", index=False)

# Benchmarking for 1x

In [5]:
# Load CSV
csv_file_1x = './all_stocks_5yr.csv'
df_1x = pd.read_csv(csv_file_1x)

# Benchmark CSV write time
start = time.time()
df_1x.to_csv(csv_file_1x, index=False)  # Writing CSV
csv_write_time_1x = time.time() - start


# Benchmark CSV Read Time
start = time.time()
df_benchmark_1x = pd.read_csv(csv_file_1x)
csv_read_time_1x = time.time() - start

# Convert to Parquet with different compression options
parquet_file_1x = './all_stocks_5yr.parquet'
start = time.time()
df_1x.to_parquet(parquet_file_1x, engine="pyarrow", compression="zstd") # Convert to Parquet with different compression options
parquet_write_time_1x = time.time() - start

# Benchmark Parquet Read Time
start = time.time()
df_parquet_1x = pd.read_parquet(parquet_file_1x, engine="pyarrow")
parquet_read_time_1x = time.time() - start


# Storage sizes
csv_size_1x = os.path.getsize(csv_file_1x) / (1024 * 1024)  # in MB
parquet_size_1x = os.path.getsize(parquet_file_1x) / (1024 * 1024)


# Print results
print(f"CSV Read Time: {csv_read_time_1x:.4f}s")
print(f"CSV Write Time: - {csv_write_time_1x:.4f}s")
print(f"Parquet Read Time: {parquet_read_time_1x:.4f}s")
print(f"Parquet Write Time: {parquet_write_time_1x:.4f}s")
print(f"CSV Size: {csv_size_1x:.2f} MB")
print(f"Parquet Size (zstd): {parquet_size_1x:.2f} MB")

CSV Read Time: 0.3362s
CSV Write Time: - 1.5488s
Parquet Read Time: 0.2060s
Parquet Write Time: 0.3799s
CSV Size: 28.80 MB
Parquet Size (zstd): 8.09 MB


# Benchmarking for 10x

In [10]:
# Load CSV
csv_file_10x = './all_stocks_5yr_10x.csv'
df_10x = pd.read_csv(csv_file_10x)

# Benchmark CSV write time
start = time.time()
df_10x.to_csv(csv_file_10x, index=False)  # Writing CSV
csv_write_time_10x = time.time() - start


# Benchmark CSV Read Time
start = time.time()
df_benchmark_10x = pd.read_csv(csv_file_10x)
csv_read_time_10x = time.time() - start

# Convert to Parquet with different compression options
parquet_file_10x = './all_stocks_5yr_10x.parquet'
start = time.time()
df_10x.to_parquet(parquet_file_10x, engine="pyarrow", compression="zstd") # Convert to Parquet with different compression options
parquet_write_time_10x = time.time() - start

# Benchmark Parquet Read Time
start = time.time()
df_parquet_10x = pd.read_parquet(parquet_file_10x, engine="pyarrow")
parquet_read_time_10x = time.time() - start


# Storage sizes
csv_size_10x = os.path.getsize(csv_file_10x) / (1024 * 1024)  # in MB
parquet_size_10x = os.path.getsize(parquet_file_10x) / (1024 * 1024)


# Print results
print(f"CSV Read Time: {csv_read_time_10x:.4f}s")
print(f"Parquet Read Time: {parquet_read_time_10x:.4f}s")
print(f"CSV Write Time: {csv_write_time_10x:.4f}s")
print(f"Parquet Write Time: {parquet_write_time_10x:.4f}s")
print(f"CSV Size: {csv_size_10x:.2f} MB")
print(f"Parquet Size (zstd): {parquet_size_10x:.2f} MB")

CSV Read Time: 3.1831s
Parquet Read Time: 0.9870s
CSV Write Time: 17.6560s
Parquet Write Time: 2.9702s
CSV Size: 288.01 MB
Parquet Size (zstd): 75.43 MB


# Benchmarking for 100x

In [11]:
# Load CSV
csv_file_100x = './all_stocks_5yr_100x.csv'
chunk_size = 10000


# Benchmark CSV write time
# Expand CSV to 100x (Using Chunking for Efficiency)**
start = time.time()
with open(csv_file_100x, "w") as f_out:
    for i in range(100):  # Repeat 100 times
        for chunk in pd.read_csv(csv_file_1x, chunksize=chunk_size):
            chunk.to_csv(f_out, mode="a", index=False, header=(f_out.tell() == 0))
csv_write_time_100x = time.time() - start


# Benchmark CSV Read Time
start = time.time()
# for chunk in pd.read_csv(csv_file_100x, chunksize=chunk_size):
#     chunks.append(chunk)
# df_100x = pd.concat(chunks, ignore_index=True)
chunks = pd.read_csv(csv_file_100x, chunksize=chunk_size)
csv_read_time_100x = time.time() - start

# Define schema based on the first chunk
first_chunk = next(chunks)
schema = pa.Table.from_pandas(first_chunk).schema


# Convert to Parquet with different compression options
parquet_file_100x = './all_stocks_5yr_100x.parquet'
start = time.time()

# Write in chunks to avoid memory issues
with pq.ParquetWriter(parquet_file_100x, schema=schema, compression='zstd') as writer:
    writer.write_table(pa.Table.from_pandas(first_chunk))
    for chunk in chunks:
        table = pa.Table.from_pandas(chunk)
        writer.write_table(table)

parquet_write_time_100x = time.time() - start

# Benchmark Parquet Read Time
start = time.time()
df_parquet_100x = pd.read_parquet(parquet_file_100x, engine="pyarrow")
parquet_read_time_100x = time.time() - start


# Storage sizes
csv_size_100x = os.path.getsize(csv_file_100x) / (1024 * 1024)  # in MB
parquet_size_100x = os.path.getsize(parquet_file_100x) / (1024 * 1024)


# Print results
print(f"CSV Read Time: {csv_read_time_100x:.4f}s")
print(f"Parquet Read Time: {parquet_read_time_100x:.4f}s")
print(f"CSV Write Time: {csv_write_time_100x:.4f}s")
print(f"Parquet Write Time: {parquet_write_time_100x:.4f}s")
print(f"CSV Size: {csv_size_100x:.2f} MB")
print(f"Parquet Size (zstd): {parquet_size_100x:.2f} MB")

CSV Read Time: 0.0409s
Parquet Read Time: 105.3366s
CSV Write Time: 297.7248s
Parquet Write Time: 157.0846s
CSV Size: 2939.09 MB
Parquet Size (zstd): 1271.86 MB


# Delete memory variables if they no longer in use

In [ ]:
del df_10x, df_parquet_100x, data_expanded_100x, data_expanded_10x, df_benchmark_10x, df_parquet_10x, original_data_1x  # Delete large DataFrames
gc.collect()



20

In [6]:
# Ensure data is sorted by date
df_1x['date'] = pd.to_datetime(df_1x['date'])
df_1x = df_1x.sort_values(by=['date', 'name'])


### ** Exponential Moving Average (EMA) - 14 Days**
df_1x['EMA_14'] = df_1x.groupby('name')['close'].transform(lambda x: x.ewm(span=14, adjust=False).mean())


### ** Relative Strength Index (RSI) - 14 Days**
def compute_rsi(series, window=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

df_1x['RSI_14'] = df_1x.groupby('name')['close'].transform(compute_rsi)


### ** MACD - Trend & Momentum Indicator**
short_ema = df_1x.groupby('name')['close'].transform(lambda x: x.ewm(span=12, adjust=False).mean())
long_ema = df_1x.groupby('name')['close'].transform(lambda x: x.ewm(span=26, adjust=False).mean())
df_1x['MACD'] = short_ema - long_ema  # MACD Line
df_1x['MACD_Signal'] = df_1x.groupby('name')['MACD'].transform(lambda x: x.ewm(span=9, adjust=False).mean())  # Signal Line


### **Bollinger Bands Calculation**
window = 20  # Standard period
std_multiplier = 2  # Default multiplier

# Calculate SMA (Middle Band)
df_1x['BB_Middle'] = df_1x.groupby('name')['close'].transform(lambda x: x.rolling(window=window).mean())

# Calculate Standard Deviation
df_1x['BB_Std'] = df_1x.groupby('name')['close'].transform(lambda x: x.rolling(window=window).std())

# Calculate Upper & Lower Bands
df_1x['BB_Upper'] = df_1x['BB_Middle'] + (df_1x['BB_Std'] * std_multiplier)
df_1x['BB_Lower'] = df_1x['BB_Middle'] - (df_1x['BB_Std'] * std_multiplier)

# Drop the temporary standard deviation column
df_1x.drop(columns=['BB_Std'], inplace=True)




In [7]:
df_1x.tail()

,date,open,high,low,close,volume,name,EMA_14,RSI_14,MACD,MACD_Signal,BB_Middle,BB_Upper,BB_Lower
614003,2018-02-07,71.27,72.860,71.12,71.79,1748941,XYL,72.526190,52.463768,0.984861,1.234707,72.2255,75.547620,68.903380
615262,2018-02-07,80.07,81.940,80.00,80.13,3561068,YUM,82.833472,38.110488,-0.324452,0.343658,83.6110,87.469109,79.752891
616521,2018-02-07,121.65,123.520,120.74,120.78,1145267,ZBH,124.216363,46.518519,0.729324,1.448836,124.2075,128.569657,119.845343
617780,2018-02-07,52.23,54.315,52.22,54.02,6427280,ZION,53.390353,55.451128,0.591228,0.801981,53.5065,55.097796,51.915204
619039,2018-02-07,72.70,75.000,72.69,73.86,4534912,ZTS,75.977092,40.749064,0.376822,1.092467,76.5855,80.570680,72.600320


In [8]:
df_1x.isnull().sum()

date              0
open             11
high              8
low               8
close             0
volume            0
name              0
EMA_14            0
RSI_14         6565
MACD              0
MACD_Signal       0
BB_Middle      9595
BB_Upper       9595
BB_Lower       9595
dtype: int64

# Handling missing values using forward-fill

In [9]:
df_1x[['RSI_14', 'BB_Middle', 'BB_Upper', 'BB_Lower']] = df_1x[['RSI_14', 'BB_Middle', 'BB_Upper', 'BB_Lower']].ffill()


In [10]:
df_1x.isnull().sum()

date              0
open             11
high              8
low               8
close             0
volume            0
name              0
EMA_14            0
RSI_14         6188
MACD              0
MACD_Signal       0
BB_Middle      9044
BB_Upper       9044
BB_Lower       9044
dtype: int64

In [11]:
df_1x[['open','high','low']] = df_1x[['open','high','low']].ffill()

In [12]:
df_1x.isnull().sum()

date              0
open              0
high              0
low               0
close             0
volume            0
name              0
EMA_14            0
RSI_14         6188
MACD              0
MACD_Signal       0
BB_Middle      9044
BB_Upper       9044
BB_Lower       9044
dtype: int64

In [13]:
df_1x.dropna(inplace=True)

In [14]:
df_1x.isnull().sum()

date           0
open           0
high           0
low            0
close          0
volume         0
name           0
EMA_14         0
RSI_14         0
MACD           0
MACD_Signal    0
BB_Middle      0
BB_Upper       0
BB_Lower       0
dtype: int64

# Saving the cleaned dataset with technical indicators introduced

In [15]:
df_1x.to_csv("all_stocks_5yr_with_indicators_cleaned.csv", index=False)

In [16]:
del df_1x
gc.collect()

203

In [17]:
df = pd.read_csv("all_stocks_5yr_with_indicators_cleaned.csv")

In [18]:
# Select Features (X) - Exclude "date", "name", and "close" (target variable)
X = df.drop(columns=["date", "name", "close"])

# Target Variable (y) - Predicting "close" price
y = df["close"]

In [19]:
# Split data into 80% training and 20% testing (without shuffling)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Random Forest Regressor Model

In [20]:
# Initialize Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train, y_train)

# Predict on test set
rf_predictions = rf_model.predict(X_test)

In [21]:
# Evaluate Performance
rf_mae = mean_absolute_error(y_test, rf_predictions)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
rf_r2 = r2_score(y_test, rf_predictions)

print(f" Random Forest - MAE: {rf_mae:.4f}, RMSE: {rf_rmse:.4f}")
print(f" Random Forest - R² Score: {rf_r2:.4f}")



 Random Forest - MAE: 0.7868, RMSE: 9.9199
 Random Forest - R² Score: 0.9940


# XGBoost Model

In [22]:
import xgboost as xgb

# Initialize XGBoost Regressor
xgb_model = xgb.XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Predict on test set
xgb_predictions = xgb_model.predict(X_test)



In [23]:
# Evaluate Performance
xgb_mae = mean_absolute_error(y_test, xgb_predictions)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
xgb_r2 = r2_score(y_test, xgb_predictions)

print(f" XGBoost - MAE: {xgb_mae:.4f}, RMSE: {xgb_rmse:.4f}")
print(f" XGBoost - R² Score: {xgb_r2:.4f}")

 XGBoost - MAE: 3.2610, RMSE: 34.0805
 XGBoost - R² Score: 0.9289


In [24]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

rf_mape = mean_absolute_percentage_error(y_test, rf_predictions)
xgb_mape = mean_absolute_percentage_error(y_test, xgb_predictions)

print(f"📌 Random Forest - MAPE: {rf_mape:.2f}%")
print(f"📌 XGBoost - MAPE: {xgb_mape:.2f}%")

📌 Random Forest - MAPE: 0.41%
📌 XGBoost - MAPE: 1.02%


In [25]:
#df['RF_Prediction'] = rf_predictions
#df['XGB_Prediction'] = xgb_predictions

print(rf_predictions.shape)
print(xgb_predictions.shape)

(122000,)
(122000,)


# Dashboarding

In [35]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [36]:
import dash
from dash import dcc, html

In [37]:
pip install jupyter-dash


Note: you may need to restart the kernel to use updated packages.


In [38]:
from jupyter_dash import JupyterDash

In [39]:
pip install dash-bootstrap-components

Note: you may need to restart the kernel to use updated packages.


In [40]:

import dash_bootstrap_components as dbc
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load Data
data = {
    "Indicator": ["Write Time", "Read Time", "Size (MB)"],
    "CSV 1x": [2.5568, 0.4831, 28.8],
    "Parquet 1x": [0.7258, 0.3468, 8.09],
    "CSV 10x": [17.656, 3.1831, 288.01],
    "Parquet 10x": [2.9702, 0.987, 75.43],
    "CSV 100x": [297.7248, 0.0409, 2929.09],
    "Parquet 100x": [157.0846, 105.3366, 1271.86],
}
dashboard_data = pd.DataFrame(data)

# Initialize Dash App
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SOLAR])  # Change theme here

# Layout
app.layout = dbc.Container([
    html.H1("Benchmarking Dashboard: CSV vs Parquet", className="text-center mt-4 mb-4"),

    dbc.Row([
        dbc.Col([
            html.Label("Select Indicator:", className="fw-bold"),
            dcc.Dropdown(
                id="indicator-dropdown",
                options=[{"label": i, "value": i} for i in df["Indicator"]],
                value="Write Time",
                clearable=False,
                className="mb-3"
            )
        ], width=6)
    ]),

    dbc.Row([
        dbc.Col(dcc.Graph(id="benchmark-chart"), width=12)
    ]),

    html.Br(),
    dbc.Row([
        dbc.Col(html.Div("📊 Data Overview", className="fw-bold fs-4 text-primary"), width=12)
    ]),
    dbc.Row([
        dbc.Col(dbc.Table.from_dataframe(df, striped=True, bordered=True, hover=True), width=12)
    ])
], fluid=True)

# Callback for updating the chart
@app.callback(
    Output("benchmark-chart", "figure"),
    Input("indicator-dropdown", "value")
)
def update_chart(indicator):
    filtered_df = dashboard_data[dashboard_data["Indicator"] == indicator].T.reset_index()
    filtered_df.columns = ["Scale", "Value"]
    filtered_df = filtered_df.iloc[1:]

    fig = px.bar(filtered_df, x="Scale", y="Value", color="Scale",
                 title=f"{indicator} Performance at Different Scales",
                 labels={"Value": "Time (s) / Storage (MB)"},
                 template="plotly_dark")  # Change template for dark theme

    return fig

# Run Server
if __name__ == "__main__":
    app.run_server(debug=True)


KeyError: 'Indicator'

In [31]:
df.head()

,date,open,high,low,close,volume,name,EMA_14,RSI_14,MACD,MACD_Signal,BB_Middle,BB_Upper,BB_Lower
0,2013-03-08,43.5000,43.5200,43.0200,43.0300,3256301,A,42.700636,56.351792,-0.476385,-0.676385,42.808500,45.407449,40.209551
1,2013-03-08,14.9900,15.2000,14.8400,14.9200,10593700,AAL,14.150636,55.440415,-0.014946,-0.190357,14.007500,15.205672,12.809328
2,2013-03-08,76.4400,77.1600,75.6302,76.8400,736386,AAP,77.310074,37.837838,-0.660033,-0.475772,77.996000,80.625720,75.366280
3,2013-03-08,61.3999,62.2042,61.2299,61.6742,97854442,AAPL,62.798035,33.187954,-1.625554,-1.408610,64.068855,69.005875,59.131835
4,2013-03-08,37.6400,37.6500,37.1600,37.3400,4114449,ABBV,37.425788,48.039216,0.293560,0.329805,37.261000,39.283129,35.238871
